### Must run step by step as there are command lines to be executed in the middle of the flow

### Input files: Price-Today.csv
### Output files: daily-sales.csv, daily-sales-prices.csv

In [3]:
import numpy as np
import os
import pandas as pd
from datetime import date, timedelta, datetime
from sqlalchemy import create_engine
from pandas.tseries.offsets import BDay

engine = create_engine("sqlite:///c:\\ruby\\port_lite\\db\\development.sqlite3")
conlite = engine.connect()

today = date.today()
print(today)

2025-08-07


In [5]:
# convert the timedelta object to a BusinessDay object
num_business_days = BDay(1)
yesterday = today - num_business_days
yesterday = yesterday.date()
print(f'Today: {today}')
print(f'Yesterday: {yesterday}')

Today: 2025-08-07
Yesterday: 2025-08-06


In [7]:
cols = 'name fm_date to_date fm_price to_price qty max_price min_price percent status'.split()

format_dict = {
    'fm_price':'{:.2f}','to_price':'{:.2f}','diff':'{:.2f}',
    'max_price':'{:.2f}','min_price':'{:.2f}',
    'volume':'{:,.2f}','beta':'{:,.2f}',
    'pct':'{:,.2f}%','percent':'{:,.2f}%',   
    'fm_date':'{:%Y-%m-%d}','to_date':'{:%Y-%m-%d}',
    'created_at':'{:%Y-%m-%d}','updated_at':'{:%Y-%m-%d}',
    
    'qty':'{:,}','available_qty':'{:,}',
    'cost':'{:.2f}','buy_target':'{:.2f}','sell_target':'{:.2f}',
}

In [9]:
# Get the user's home directory
user_path = os.path.expanduser('~')
# Get the current working directory
current_path = os.getcwd()
# Derive the base directory (base_dir) by removing the last folder ('Daily')
base_path = os.path.dirname(current_path)
#C:\Users\PC1\OneDrive\A5\Data
dat_path = os.path.join(base_path, "Data")
#C:\Users\PC1\OneDrive\Imports\santisoontarinka@gmail.com - Google Drive\Data>
god_path = os.path.join(user_path, "OneDrive","Imports","santisoontarinka@gmail.com - Google Drive","Data")
#C:\Users\PC1\iCloudDrive\data
icd_path = os.path.join(user_path, "iCloudDrive", "Data")
#C:\Users\PC1\OneDrive\Documents\obsidian-git-sync\Data
osd_path = os.path.join(user_path, "OneDrive","Documents","obsidian-git-sync","Data")

In [11]:
print("User path:", user_path)
print(f"Current path: {current_path}")
print(f"Base path: {base_path}")
print(f"Data path : {dat_path}") 
print(f"Google Drive path : {god_path}")
print(f"iCloudDrive path : {icd_path}") 
print(f"OSD path : {osd_path}") 

User path: C:\Users\PC1
Current path: C:\Users\PC1\OneDrive\A5\Daily
Base path: C:\Users\PC1\OneDrive\A5
Data path : C:\Users\PC1\OneDrive\A5\Data
Google Drive path : C:\Users\PC1\OneDrive\Imports\santisoontarinka@gmail.com - Google Drive\Data
iCloudDrive path : C:\Users\PC1\iCloudDrive\Data
OSD path : C:\Users\PC1\OneDrive\Documents\obsidian-git-sync\Data


In [13]:
sql = """
SELECT DISTINCT a.name
FROM sales a 
WHERE to_date = '%s'
ORDER BY a.name
"""
sql = sql % yesterday
print(sql)
tmp = pd.read_sql(sql, conlite)
tmp.shape


SELECT DISTINCT a.name
FROM sales a 
WHERE to_date = '2025-08-06'
ORDER BY a.name



(11, 1)

In [15]:
sql = """
SELECT a.name,fm_date,to_date,fm_price,to_price,
a.qty,a.max_price,a.min_price,t.status,t.market
FROM sales a 
JOIN stocks t ON a.name = t.name 
WHERE to_date = '%s' AND t.status IN ("B","I", "O", "S") 
ORDER BY t.status, a.name
"""
sql = sql % yesterday
print(sql)


SELECT a.name,fm_date,to_date,fm_price,to_price,
a.qty,a.max_price,a.min_price,t.status,t.market
FROM sales a 
JOIN stocks t ON a.name = t.name 
WHERE to_date = '2025-08-06' AND t.status IN ("B","I", "O", "S") 
ORDER BY t.status, a.name



### This statement causes program to hang when there is no data

In [18]:
df = pd.read_sql(sql, conlite)
#df.set_index(["name"], inplace=True)
df['fm_date'] = pd.to_datetime(df['fm_date'])
df['to_date'] = pd.to_datetime(df['to_date'])
df.eval('diff = to_price - fm_price',inplace=True)
df['percent'] = round(df['diff']/df['fm_price']*100,2)
#df.style.format(format_dict)
df

,name,fm_date,to_date,fm_price,to_price,qty,max_price,min_price,status,market,diff,percent
0,DIF,2025-07-31,2025-08-06,8.15,8.35,85607088,8.35,8.15,B,SET,0.20,2.45
1,GVREIT,2025-08-06,2025-08-06,6.60,6.55,1195892,6.60,6.55,B,SET,-0.05,-0.76
2,JMT,2025-08-05,2025-08-06,10.50,12.40,86843944,12.40,10.50,B,SET50,1.90,18.10
3,SENA,2025-08-05,2025-08-06,1.74,1.79,445936,1.82,1.72,B,SET,0.05,2.87
4,WHAIR,2025-08-05,2025-08-06,5.45,5.55,1631700,5.55,5.40,B,SET,0.10,1.83
5,WHART,2025-08-05,2025-08-06,9.20,9.40,4356956,9.40,9.20,B,SET,0.20,2.17
6,AIMIRT,2025-08-05,2025-08-06,9.80,9.85,1565872,9.90,9.55,S,SET,0.05,0.51
7,IVL,2025-08-06,2025-08-06,22.50,23.70,36456644,23.70,22.40,S,SET50,1.20,5.33
8,MCS,2025-08-01,2025-08-06,8.30,8.45,1098712,8.45,8.30,S,SET,0.15,1.81
9,NER,2025-08-05,2025-08-06,4.42,4.52,96832194,4.56,4.22,S,SET,0.10,2.26


In [20]:
df.shape

(11, 12)

### IF the above count not equal number of orders, there must be something incorrect

### Create daily-sales from sales

In [24]:
df[cols].style.format(format_dict)

,name,fm_date,to_date,fm_price,to_price,qty,max_price,min_price,percent,status
0,DIF,2025-07-31,2025-08-06,8.15,8.35,"85,607,088",8.35,8.15,2.45%,B
1,GVREIT,2025-08-06,2025-08-06,6.60,6.55,"1,195,892",6.60,6.55,-0.76%,B
2,JMT,2025-08-05,2025-08-06,10.50,12.40,"86,843,944",12.40,10.50,18.10%,B
3,SENA,2025-08-05,2025-08-06,1.74,1.79,"445,936",1.82,1.72,2.87%,B
4,WHAIR,2025-08-05,2025-08-06,5.45,5.55,"1,631,700",5.55,5.40,1.83%,B
5,WHART,2025-08-05,2025-08-06,9.20,9.40,"4,356,956",9.40,9.20,2.17%,B
6,AIMIRT,2025-08-05,2025-08-06,9.80,9.85,"1,565,872",9.90,9.55,0.51%,S
7,IVL,2025-08-06,2025-08-06,22.50,23.70,"36,456,644",23.70,22.40,5.33%,S
8,MCS,2025-08-01,2025-08-06,8.30,8.45,"1,098,712",8.45,8.30,1.81%,S
9,NER,2025-08-05,2025-08-06,4.42,4.52,"96,832,194",4.56,4.22,2.26%,S


In [26]:
file_name = "daily-sales.csv"
output_file = os.path.join(dat_path, file_name)
god_file = os.path.join(god_path, file_name)
icd_file = os.path.join(icd_path, file_name)
osd_file = os.path.join(osd_path, file_name)

In [28]:
print(f"Output file : {output_file}") 
print(f"icd_file : {icd_file}") 
print(f"god_file : {god_file}") 
print(f"osd_file : {osd_file}") 

Output file : C:\Users\PC1\OneDrive\A5\Data\daily-sales.csv
icd_file : C:\Users\PC1\iCloudDrive\Data\daily-sales.csv
god_file : C:\Users\PC1\OneDrive\Imports\santisoontarinka@gmail.com - Google Drive\Data\daily-sales.csv
osd_file : C:\Users\PC1\OneDrive\Documents\obsidian-git-sync\Data\daily-sales.csv


In [30]:
df[cols].sort_values(['status','percent'],ascending=[True,True]).to_csv(output_file, header=True, index=False)
df[cols].sort_values(['status','percent'],ascending=[True,True]).to_csv(god_file, header=True, index=False)
df[cols].sort_values(['status','percent'],ascending=[True,True]).to_csv(icd_file, header=True, index=False)
df[cols].sort_values(['status','percent'],ascending=[True,True]).to_csv(osd_file, header=True, index=False)

In [32]:
sales = df[cols]
sales.shape

(11, 10)

In [34]:
file_name = "Price-Today.csv"
input_file = os.path.join(dat_path, file_name)
print(f"Input file: {input_file}")

Input file: C:\Users\PC1\OneDrive\A5\Data\Price-Today.csv


In [36]:
prices = pd.read_csv(input_file)
prices.shape

(169, 10)

In [38]:
df_merge = pd.merge(sales,prices,on='name', how='inner')
df_merge.shape

(11, 19)

In [40]:
colu = 'name fm_date to_date fm_price to_price qty max_price min_price percent status \
price change volume date'.split()
df_merge[colu]

,name,fm_date,to_date,fm_price,to_price,qty,max_price,min_price,percent,status,price,change,volume,date
0,DIF,2025-07-31,2025-08-06,8.15,8.35,85607088,8.35,8.15,2.45,B,8.40,0.05,8257569,2025-08-07
1,GVREIT,2025-08-06,2025-08-06,6.60,6.55,1195892,6.60,6.55,-0.76,B,6.55,0.00,681610,2025-08-07
2,JMT,2025-08-05,2025-08-06,10.50,12.40,86843944,12.40,10.50,18.10,B,12.10,-0.30,37818799,2025-08-07
3,SENA,2025-08-05,2025-08-06,1.74,1.79,445936,1.82,1.72,2.87,B,1.81,0.02,260865,2025-08-07
4,WHAIR,2025-08-05,2025-08-06,5.45,5.55,1631700,5.55,5.40,1.83,B,5.55,0.00,677506,2025-08-07
5,WHART,2025-08-05,2025-08-06,9.20,9.40,4356956,9.40,9.20,2.17,B,9.30,-0.10,1209113,2025-08-07
6,AIMIRT,2025-08-05,2025-08-06,9.80,9.85,1565872,9.90,9.55,0.51,S,9.85,0.00,656505,2025-08-07
7,IVL,2025-08-06,2025-08-06,22.50,23.70,36456644,23.70,22.40,5.33,S,23.40,-0.30,14449557,2025-08-07
8,MCS,2025-08-01,2025-08-06,8.30,8.45,1098712,8.45,8.30,1.81,S,8.65,0.20,865774,2025-08-07
9,NER,2025-08-05,2025-08-06,4.42,4.52,96832194,4.56,4.22,2.26,S,4.54,0.02,7354598,2025-08-07


In [42]:
file_name = "daily-sales-prices.csv"
output_file = os.path.join(dat_path, file_name)
god_file = os.path.join(god_path, file_name)
icd_file = os.path.join(icd_path, file_name)
osd_file = os.path.join(osd_path, file_name)

In [44]:
print(f"Output file: {output_file}") 
print(f"icd_file : {icd_file}") 
print(f"god_file : {god_file}") 
print(f"osd_file : {osd_file}") 

Output file: C:\Users\PC1\OneDrive\A5\Data\daily-sales-prices.csv
icd_file : C:\Users\PC1\iCloudDrive\Data\daily-sales-prices.csv
god_file : C:\Users\PC1\OneDrive\Imports\santisoontarinka@gmail.com - Google Drive\Data\daily-sales-prices.csv
osd_file : C:\Users\PC1\OneDrive\Documents\obsidian-git-sync\Data\daily-sales-prices.csv


In [46]:
df_merge[colu].sort_values(['name'],ascending=[True]).to_csv(output_file, header=True, index=False)
df_merge[colu].sort_values(['name'],ascending=[True]).to_csv(god_file, header=True, index=False)
df_merge[colu].sort_values(['name'],ascending=[True]).to_csv(icd_file, header=True, index=False)
df_merge[colu].sort_values(['name'],ascending=[True]).to_csv(osd_file, header=True, index=False)

### Add or update sells record depends on trend

In [49]:
os.chdir(base_path)
%pwd

'C:\\Users\\PC1\\OneDrive\\A5'

In [51]:
!ruby ruby\\daily-out-new.rb

Name      From Date    To Date   From     To     Pct      Shares    Max    Min S Action
---------------------------------------------------------------------------------------
AIMIRT   2025-08-05 2025-08-07   9.80   9.85   0.51%   1,565,872   9.90   9.55 S Update
DIF      2025-07-31 2025-08-07   8.15   8.40   3.07%  85,607,088   8.40   8.15 B Update
GVREIT   2025-08-06 2025-08-07   6.60   6.55  -0.76%   1,195,892   6.60   6.55 B Update
IVL      2025-08-07 2025-08-07  23.70  23.40  -1.27%  36,456,644  23.70  22.40 S Insert
JMT      2025-08-07 2025-08-07  12.40  12.10  -2.42%  86,843,944  12.40  10.50 B Insert
MCS      2025-08-01 2025-08-07   8.30   8.65   4.22%   1,098,712   8.65   8.30 S Update
NER      2025-08-05 2025-08-07   4.42   4.54   2.71%  96,832,194   4.56   4.22 S Update
PTG      2025-08-07 2025-08-07   6.85   6.80  -0.73%  50,285,196   6.85   6.55 S Insert
SENA     2025-08-05 2025-08-07   1.74   1.81   4.02%     445,936   1.82   1.72 B Update
WHAIR    2025-08-05 2025-08-07  

In [53]:
os.chdir(current_path)
%pwd

'C:\\Users\\PC1\\OneDrive\\A5\\Daily'



### cd\ruby\port_lite


### rails runner db/ins_sales.rb



In [55]:
sql = """
SELECT a.name,fm_date,to_date,fm_price,to_price,
a.qty,a.max_price,a.min_price,t.status,t.market
FROM sales a 
JOIN stocks t ON a.name = t.name 
WHERE to_date = '%s' AND t.status IN ("B","I", "O", "S") 
ORDER BY t.status, a.name
"""
sql = sql % today
print(sql)


SELECT a.name,fm_date,to_date,fm_price,to_price,
a.qty,a.max_price,a.min_price,t.status,t.market
FROM sales a 
JOIN stocks t ON a.name = t.name 
WHERE to_date = '2025-08-07' AND t.status IN ("B","I", "O", "S") 
ORDER BY t.status, a.name



In [57]:
df = pd.read_sql(sql, conlite)
df.shape

(11, 10)

In [59]:
#df.set_index(["name"], inplace=True)
df['fm_date'] = pd.to_datetime(df['fm_date'])
df['to_date'] = pd.to_datetime(df['to_date'])
df.eval('diff = to_price - fm_price',inplace=True)
df['percent'] = round(df['diff']/df['fm_price']*100,2)
df.style.format(format_dict)

,name,fm_date,to_date,fm_price,to_price,qty,max_price,min_price,status,market,diff,percent
0,DIF,2025-07-31,2025-08-07,8.15,8.40,"171,214,176",8.40,8.15,B,SET,0.25,3.07%
1,GVREIT,2025-08-06,2025-08-07,6.60,6.55,"2,391,784",6.60,6.55,B,SET,-0.05,-0.76%
2,JMT,2025-08-07,2025-08-07,12.40,12.10,"86,843,944",12.40,10.50,B,SET50,-0.30,-2.42%
3,SENA,2025-08-05,2025-08-07,1.74,1.81,"891,872",1.82,1.72,B,SET,0.07,4.02%
4,WHAIR,2025-08-05,2025-08-07,5.45,5.55,"3,263,400",5.55,5.40,B,SET,0.10,1.83%
5,WHART,2025-08-07,2025-08-07,9.40,9.30,"4,356,956",9.40,9.20,B,SET,-0.10,-1.06%
6,AIMIRT,2025-08-05,2025-08-07,9.80,9.85,"3,131,744",9.90,9.55,S,SET,0.05,0.51%
7,IVL,2025-08-07,2025-08-07,23.70,23.40,"36,456,644",23.70,22.40,S,SET50,-0.30,-1.27%
8,MCS,2025-08-01,2025-08-07,8.30,8.65,"2,197,424",8.65,8.30,S,SET,0.35,4.22%
9,NER,2025-08-05,2025-08-07,4.42,4.54,"193,664,388",4.56,4.22,S,SET,0.12,2.71%


In [61]:
df_sort = df.sort_values(by=['percent'],ascending=[True]).copy()

In [63]:
df_sort.query('percent <= -5.00').style.format(format_dict)

,name,fm_date,to_date,fm_price,to_price,qty,max_price,min_price,status,market,diff,percent


In [65]:
df_sort.query('percent >= 5.00').style.format(format_dict)

,name,fm_date,to_date,fm_price,to_price,qty,max_price,min_price,status,market,diff,percent


In [67]:
conlite.close()

In [69]:
# Get the current time
current_time = datetime.now()
# Format the time to remove milliseconds
formatted_time = current_time.strftime("%Y-%m-%d %H:%M:%S")
print(formatted_time)

2025-08-07 22:36:22
